In [76]:
import pandas as pd
import numpy as np
import unicodedata
import re
import uuid
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

import warnings
warnings.filterwarnings("ignore")

## Extracción

In [2]:
exportaciones_septiembre = pd.read_excel("./Datasets/09_Exportaciones_2025_Septiembre.xlsx")
exportaciones_agosto = pd.read_excel("./Datasets/08_Exportaciones_2025_Agosto.xlsx")
exportaciones_julio = pd.read_excel("./Datasets/07_Exportaciones_2025_Julio.xlsx")
exportaciones_junio = pd.read_excel("./Datasets/06_Exportaciones_2025_Junio.xlsx")
exportaciones_mayo = pd.read_excel("./Datasets/05_Exportaciones_2025_Mayo.xlsx")
exportaciones_abril = pd.read_excel("./Datasets/04_Exportaciones_2025_Abril.xlsx")
exportaciones_marzo = pd.read_excel("./Datasets/03_Exportaciones_2025_Marzo.xlsx")
exportaciones_febrero = pd.read_excel("./Datasets/02_Exportaciones_2025_Febrero.xlsx")
exportaciones_enero = pd.read_excel("./Datasets/01_Exportaciones_2025_Enero.xlsx")

## Transformación

In [3]:
total_exports = pd.concat([
    exportaciones_enero,
    exportaciones_febrero,
    exportaciones_marzo,
    exportaciones_abril,
    exportaciones_mayo,
    exportaciones_junio,
    exportaciones_julio,
    exportaciones_agosto,
    exportaciones_septiembre
])
total_exports

,FECHA_PROCESO,NUMERO_SERIE,OFICINA,COD_ADUANA_DESPACHO,ADUANA_DESPACHO,TIPO_IDENT,NIT_EXPORTADOR,TIPO_USUARIO,COD_USUARIO,CLASE_EXPORTADOR,...,ADUANA_SALIDA,FECHA_SOLICITUD_AUTO_EMBARQUE,NUMERO_FORMULARIO,FECHA_DECLARACION_EXPORTACION,RAZON_SOCIAL_EXPORTADOR,DIREC_EXPORTADOR,NIT_DECLARANTE,RAZON_SOCIAL_DECLARANTE,RAZON_SOCIAL_DESTINATARIO,DOMICILIO_DESTINATARIO
0,202501,1,99,3,Aduanas de Bogotá - Aeropuerto El Dorado,2,900647398.0,36,0,2,...,Aduanas de Bogotá - Aeropuerto El Dorado,202501,6007749406121,20250124,THE PREMIER TRADING CI SAS,CL 20 NORTE 5 B N 38 OF 301,890309238.0,AGENCIA DE ADUANAS FEDEGAL SAS NIVEL II,VR TRADING LTD.,"ROOM 1402, HIP KWAN COMMERCIAL BLDG, NO. 38 PI..."
1,202501,1,99,87,Aduanas de Barranquilla,2,860026753.0,36,0,2,...,Aduanas de Barranquilla,202412,6007748464329,20250110,ACESCO COLOMBIA S.A.S.,AV CL 116 7 15 IN 2 P 17 OF 1701,860026753.0,ACESCO COLOMBIA S.A.S.,BARBADOS STEEL WORKS LTD,NEWTON INDUSTRIAL PARK-BARBADOS
2,202501,1,99,87,Aduanas de Barranquilla,2,860001899.0,36,0,2,...,Aduanas de Barranquilla,202501,6007749290165,20250123,CORPACERO S. A. S.,CL 25 G 100 24,830003079.0,AGENCIA DE ADUANAS PROFESIONAL S.A.S NIVEL 1 -...,PANEL REY S.A,"SERAFIN PEÑA SUR 938 MONTERREY CENTRO, C.P. 64..."
3,202501,4,99,48,Aduanas de Cartagena,2,901436775.0,36,0,2,...,Aduanas de Cartagena,202501,6007748391103,20250108,FABRICA DE LICORES Y ALCOHOLES DE ANTIOQUIA,AC 24 96 10 BG 2 OF 303,901480825.0,AGENCIA DE ADUANAS ELITE WORLD WIDE S.A.S. NIV...,MARKETING GLOBAL BRANDS CORP./EEUU,1015 PARK CENTRE BOULEVARD MIAMI GARDENS FL
4,202501,1,99,48,Aduanas de Cartagena,2,860402328.0,36,0,2,...,Aduanas de Cartagena,202501,6007748391128,20250107,INTECPLAST INYECCION TECNICA DE PLASTICOS S. A...,AC 24 95 12 BG 16,860066749.0,AGENCIA DE ADUANAS ADUANAMIENTOS LTDA NI...,COMMERSERVICE INC,103 PARKVIEW RD SEBRING FL 33870 USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121421,202509,41,99,89,Aduanas de Cúcuta,2,890807976.0,36,0,2,...,Aduanas de Cúcuta,202508,6007762602181,20250905,BELLOTA COLOMBIA SOCIEDAD POR ACCIONES SIMPLIF...,BRR BOSQUE TV 54 21 A 120 CENTRO EMPR...,890403077.0,AGENCIA DE ADUANAS HUBEMAR S.A.S NIVEL 1,FEBECA C.A.,AV. FUNDO LA UNION LOCAL PARCELA NRO L13 Y L19...
121422,202509,40,99,89,Aduanas de Cúcuta,2,890807976.0,36,0,2,...,Aduanas de Cúcuta,202508,6007762602181,20250905,BELLOTA COLOMBIA SOCIEDAD POR ACCIONES SIMPLIF...,BRR BOSQUE TV 54 21 A 120 CENTRO EMPR...,890403077.0,AGENCIA DE ADUANAS HUBEMAR S.A.S NIVEL 1,FEBECA C.A.,AV. FUNDO LA UNION LOCAL PARCELA NRO L13 Y L19...
121423,202509,2,99,39,Impuestos y Aduanas de Maicao,2,900617753.0,36,0,2,...,Impuestos y Aduanas de Maicao,202509,6007762909984,20250919,SUMMUM SAS,CR 53 74 86 OF 303,800239422.0,AGENCIA DE ADUANAS ISASO S. A. NIVEL 1.,"INVERSIONES CANAIMA 48, C.A",CALLE LA FACULTAD CON CALLE RAZETTI QTA LONTA...
121424,202509,1,99,39,Impuestos y Aduanas de Maicao,2,900617753.0,36,0,2,...,Impuestos y Aduanas de Maicao,202509,6007762909984,20250919,SUMMUM SAS,CR 53 74 86 OF 303,800239422.0,AGENCIA DE ADUANAS ISASO S. A. NIVEL 1.,"INVERSIONES CANAIMA 48, C.A",CALLE LA FACULTAD CON CALLE RAZETTI QTA LONTA...


### Eliminación de columnas

In [4]:
cols_keep = [
    "UNIDAD_FISICA",
    "CANTIDAD_UNIDADES_FISICAS",
    "PESO_BRUTO_KGS",
    "PESO_NETO_KGS",
    "VALOR_FOB_USD",
    "VALOR_FOB_PESOS",
    "VLR_SERIE_AGREGADO_NAL_USD",
    "VALOR_SERIE_FLETES_USD",
    "VALOR_SERIE_SEGUROS_USD",
    "VLR_SERIE_OTROS_GASTOS_USD",
    "COD_MONEDA_TRANSACCION",
    "COD_MODO_TRANSPORTE",
    "COD_MODALIDAD_EXPORTACION",
    "COD_PAIS_DESTINO",
    "NIT_EXPORTADOR",
    "COD_ADUANA_DESPACHO",
    "FECHA_DECLARACION_EXPORTACION",
    "MODO_TRANSPORTE",
    "MODALIDAD_EXPORTACION",
    "PAIS_DESTINO_FINAL",
    "CIUDAD_DESTINATARIO",
    "TIPO_USUARIO",
    "COD_USUARIO",
    "CLASE_EXPORTADOR",
    "COD_DPTO_EXPORTADOR",
    "RAZON_SOCIAL_EXPORTADOR",
    "DIREC_EXPORTADOR",
    "ADUANA_DESPACHO"
]

In [34]:
total_exports_clean = total_exports[cols_keep].copy()
total_exports_clean

,UNIDAD_FISICA,CANTIDAD_UNIDADES_FISICAS,PESO_BRUTO_KGS,PESO_NETO_KGS,VALOR_FOB_USD,VALOR_FOB_PESOS,VLR_SERIE_AGREGADO_NAL_USD,VALOR_SERIE_FLETES_USD,VALOR_SERIE_SEGUROS_USD,VLR_SERIE_OTROS_GASTOS_USD,...,MODALIDAD_EXPORTACION,PAIS_DESTINO_FINAL,CIUDAD_DESTINATARIO,TIPO_USUARIO,COD_USUARIO,CLASE_EXPORTADOR,COD_DPTO_EXPORTADOR,RAZON_SOCIAL_EXPORTADOR,DIREC_EXPORTADOR,ADUANA_DESPACHO
0,Quilate,145.55,0.03,0.02,108800.00,4.619267e+08,0.00,0.00,0.0,0.00,...,Las demás exportaciones definitivas no incluid...,"China, región administrativa especial de Hong ...",KOWLOON,36,0,2,76001,THE PREMIER TRADING CI SAS,CL 20 NORTE 5 B N 38 OF 301,Aduanas de Bogotá - Aeropuerto El Dorado
1,Kilogramo,42734.00,43119.00,42734.00,44379.79,1.917735e+08,0.00,6260.00,0.0,0.00,...,Las demás exportaciones definitivas no incluid...,Barbados,CHRIST CHURCH,36,0,2,11001,ACESCO COLOMBIA S.A.S.,AV CL 116 7 15 IN 2 P 17 OF 1701,Aduanas de Barranquilla
2,Kilogramo,1014460.00,1017700.00,1014460.00,999243.10,4.274772e+09,294083.13,95694.01,119.0,0.00,...,Exportación definitiva de mercancías de fabric...,México,MONTERREY,36,0,2,11001,CORPACERO S. A. S.,CL 25 G 100 24,Aduanas de Barranquilla
3,Litro,189.00,196.07,189.00,927.36,4.026885e+06,0.00,0.00,0.0,0.00,...,Exportación definitiva de mercancías que resul...,Estados Unidos de América,MIAMI,36,0,2,11001,FABRICA DE LICORES Y ALCOHOLES DE ANTIOQUIA,AC 24 96 10 BG 2 OF 303,Aduanas de Cartagena
4,Unidades o artículos,116831.00,2473.16,1967.96,13301.52,5.793490e+07,0.00,0.00,0.0,0.00,...,Exportación definitiva de mercancías de fabric...,Estados Unidos de América,MIAMI,36,0,2,11001,INTECPLAST INYECCION TECNICA DE PLASTICOS S. A...,AC 24 95 12 BG 16,Aduanas de Cartagena
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121421,Unidades o artículos,144.00,45.75,44.00,343.35,1.370341e+06,0.00,16.65,0.0,0.00,...,Exportación definitiva de mercancías de fabric...,Venezuela (República Bolivariana de),CARABOBO,36,0,2,13001,BELLOTA COLOMBIA SOCIEDAD POR ACCIONES SIMPLIF...,BRR BOSQUE TV 54 21 A 120 CENTRO EMPR...,Aduanas de Cúcuta
121422,Unidades o artículos,192.00,57.19,55.00,423.00,1.688231e+06,0.00,20.52,0.0,0.00,...,Exportación definitiva de mercancías de fabric...,Venezuela (República Bolivariana de),CARABOBO,36,0,2,13001,BELLOTA COLOMBIA SOCIEDAD POR ACCIONES SIMPLIF...,BRR BOSQUE TV 54 21 A 120 CENTRO EMPR...,Aduanas de Cúcuta
121423,Unidades o artículos,1.00,22.45,14.88,268.40,1.044734e+06,0.00,0.00,0.0,24.16,...,Exportación definitiva de mercancías de fabric...,Venezuela (República Bolivariana de),CARACAS,36,0,2,8001,SUMMUM SAS,CR 53 74 86 OF 303,Impuestos y Aduanas de Maicao
121424,Unidades o artículos,219.00,1195.34,791.99,14281.30,5.558925e+07,0.00,0.00,0.0,1285.32,...,Exportación definitiva de mercancías de fabric...,Venezuela (República Bolivariana de),CARACAS,36,0,2,8001,SUMMUM SAS,CR 53 74 86 OF 303,Impuestos y Aduanas de Maicao


### Limpiar ciudades

In [ ]:
def limpiar_ciudad(nombre):
    nombre = str(nombre).upper().strip()

    # Eliminar acentos y caracteres especiales
    nombre = ''.join(
        c for c in unicodedata.normalize('NFD', nombre)
        if unicodedata.category(c) != 'Mn'
    )

    # Quitar codificaciones rotas tipo &#65533
    nombre = re.sub(r'&#\d+;?', '', nombre)

    # Eliminar paréntesis y texto dentro
    nombre = re.sub(r'\(.*?\)', '', nombre)

    # Eliminar comas, puntos, y sufijos administrativos como DC o D.C.
    nombre = re.sub(r',?\s*(D\.?C\.?|DC)$', '', nombre)
    nombre = re.sub(r'[^A-Z\s]', '', nombre)
    
    # Limpiar espacios repetidos
    nombre = re.sub(r'\s+', ' ', nombre).strip()

    # Homologar equivalencias comunes
    equivalencias = {
        "BOGOT": "BOGOTA",
        "BOGOTA DC": "BOGOTA",
        "SANTA FE DE BOGOTA": "BOGOTA",
        "MEDELLIN": "MEDELLIN",
        "MEDELLN": "MEDELLIN",
        "NUNCHIA": "NUNCHIA",
        "COTA": "COTA",
        "FUNZA": "FUNZA",
        "SANTA MARTA": "SANTA MARTA",
        "CARTAGENA": "CARTAGENA",
        "PEREIRA": "PEREIRA",
        "BARRANQUILLA": "BARRANQUILLA",
        "MOSQUERA": "MOSQUERA",
        "MACEIO": "MACEO",
    }

    # Reemplazo por equivalencia si existe
    if nombre in equivalencias:
        nombre = equivalencias[nombre]

    return nombre

In [35]:
total_exports_clean["CIUDAD_DESTINATARIO"] = total_exports_clean["CIUDAD_DESTINATARIO"].fillna("DESCONOCIDO")
total_exports_clean["CIUDAD_DESTINATARIO"] = total_exports_clean["CIUDAD_DESTINATARIO"].apply(limpiar_ciudad)
total_exports_clean.loc[total_exports_clean["COD_PAIS_DESTINO"] == "XCF", ["COD_PAIS_DESTINO", "PAIS_DESTINO_FINAL"]] = ["COL", "Colombia"]
total_exports_clean[total_exports_clean["COD_PAIS_DESTINO"] == "COL"]["CIUDAD_DESTINATARIO"].unique()

array(['SITIONUEVO', 'SONSON', 'BOGOTA', 'RIONEGRO', 'MEDELLIN',
       'CARTAGENA', 'BARRANQUILLA', 'MONTENEGRO', 'PALMIRA', 'FUNZA',
       'SANTA MARTA', 'VILLA RICA', 'PUERTO TEJADA', 'CAUCA', 'COTA',
       'NUNCHA', 'GALAPA', 'NUNCHIA', 'MOSQUERA', 'MACEO', 'TURBACO',
       'PUERTO LOPEZ', 'PEREIRA', 'C', 'DESCONOCIDO'], dtype=object)

### Código ciudades

In [53]:
total_exports_clean["ID_DESTINO"] = total_exports_clean["COD_PAIS_DESTINO"] + "_" + total_exports_clean["CIUDAD_DESTINATARIO"]
total_exports_clean["ID_DESTINO"] = total_exports_clean["ID_DESTINO"].str.replace(" ", "_")
total_exports_clean["ID_DESTINO"] = total_exports_clean["ID_DESTINO"].str.replace("-", "_")
total_exports_clean["ID_DESTINO"] = total_exports_clean["ID_DESTINO"].str.replace(".", "_")
total_exports_clean["ID_DESTINO"] = total_exports_clean["ID_DESTINO"].str.upper()
total_exports_clean["ID_DESTINO"].value_counts()

ID_DESTINO
USA_MIAMI                   94950
ECU_QUITO                   77955
PER_LIMA                    66528
CRI_SAN_JOSE                32595
ECU_GUAYAQUIL               24324
                            ...  
CHE_ORPUND                      1
USA_LUPTON_CITY                 1
USA_EERFIELD_BEACH              1
CRI_CARTAGENA_COSTA_RICA        1
FRA_PUGETSURARGENS              1
Name: count, Length: 10193, dtype: int64

### Fechas

In [36]:

total_exports_clean["FECHA_DECLARACION_EXPORTACION"] = pd.to_datetime(total_exports_clean["FECHA_DECLARACION_EXPORTACION"], format="%Y%m%d")
total_exports_clean["AÑO"] = total_exports_clean["FECHA_DECLARACION_EXPORTACION"].dt.year
total_exports_clean["MES"] = total_exports_clean["FECHA_DECLARACION_EXPORTACION"].dt.month
total_exports_clean["DIA"] = total_exports_clean["FECHA_DECLARACION_EXPORTACION"].dt.day
total_exports_clean

,UNIDAD_FISICA,CANTIDAD_UNIDADES_FISICAS,PESO_BRUTO_KGS,PESO_NETO_KGS,VALOR_FOB_USD,VALOR_FOB_PESOS,VLR_SERIE_AGREGADO_NAL_USD,VALOR_SERIE_FLETES_USD,VALOR_SERIE_SEGUROS_USD,VLR_SERIE_OTROS_GASTOS_USD,...,TIPO_USUARIO,COD_USUARIO,CLASE_EXPORTADOR,COD_DPTO_EXPORTADOR,RAZON_SOCIAL_EXPORTADOR,DIREC_EXPORTADOR,ADUANA_DESPACHO,AÑO,MES,DIA
0,Quilate,145.55,0.03,0.02,108800.00,4.619267e+08,0.00,0.00,0.0,0.00,...,36,0,2,76001,THE PREMIER TRADING CI SAS,CL 20 NORTE 5 B N 38 OF 301,Aduanas de Bogotá - Aeropuerto El Dorado,2025,1,24
1,Kilogramo,42734.00,43119.00,42734.00,44379.79,1.917735e+08,0.00,6260.00,0.0,0.00,...,36,0,2,11001,ACESCO COLOMBIA S.A.S.,AV CL 116 7 15 IN 2 P 17 OF 1701,Aduanas de Barranquilla,2025,1,10
2,Kilogramo,1014460.00,1017700.00,1014460.00,999243.10,4.274772e+09,294083.13,95694.01,119.0,0.00,...,36,0,2,11001,CORPACERO S. A. S.,CL 25 G 100 24,Aduanas de Barranquilla,2025,1,23
3,Litro,189.00,196.07,189.00,927.36,4.026885e+06,0.00,0.00,0.0,0.00,...,36,0,2,11001,FABRICA DE LICORES Y ALCOHOLES DE ANTIOQUIA,AC 24 96 10 BG 2 OF 303,Aduanas de Cartagena,2025,1,8
4,Unidades o artículos,116831.00,2473.16,1967.96,13301.52,5.793490e+07,0.00,0.00,0.0,0.00,...,36,0,2,11001,INTECPLAST INYECCION TECNICA DE PLASTICOS S. A...,AC 24 95 12 BG 16,Aduanas de Cartagena,2025,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121421,Unidades o artículos,144.00,45.75,44.00,343.35,1.370341e+06,0.00,16.65,0.0,0.00,...,36,0,2,13001,BELLOTA COLOMBIA SOCIEDAD POR ACCIONES SIMPLIF...,BRR BOSQUE TV 54 21 A 120 CENTRO EMPR...,Aduanas de Cúcuta,2025,9,5
121422,Unidades o artículos,192.00,57.19,55.00,423.00,1.688231e+06,0.00,20.52,0.0,0.00,...,36,0,2,13001,BELLOTA COLOMBIA SOCIEDAD POR ACCIONES SIMPLIF...,BRR BOSQUE TV 54 21 A 120 CENTRO EMPR...,Aduanas de Cúcuta,2025,9,5
121423,Unidades o artículos,1.00,22.45,14.88,268.40,1.044734e+06,0.00,0.00,0.0,24.16,...,36,0,2,8001,SUMMUM SAS,CR 53 74 86 OF 303,Impuestos y Aduanas de Maicao,2025,9,19
121424,Unidades o artículos,219.00,1195.34,791.99,14281.30,5.558925e+07,0.00,0.00,0.0,1285.32,...,36,0,2,8001,SUMMUM SAS,CR 53 74 86 OF 303,Impuestos y Aduanas de Maicao,2025,9,19


### NIT

In [91]:
total_exports_clean[total_exports_clean["NIT_EXPORTADOR"] == 0]["NIT_EXPORTADOR"].value_counts()

NIT_EXPORTADOR
0.0    1646
Name: count, dtype: int64

In [92]:
total_exports_clean["NIT_EXPORTADOR"].isna().sum()

np.int64(2881)

In [94]:
total_exports_clean["NIT_EXPORTADOR"] = total_exports_clean["NIT_EXPORTADOR"].fillna(0)
total_exports_clean.loc[total_exports_clean["NIT_EXPORTADOR"] == 0, "NIT_EXPORTADOR"] = 9999999999

total_exports_clean[total_exports_clean["NIT_EXPORTADOR"] == 9999999999]["NIT_EXPORTADOR"].value_counts()

NIT_EXPORTADOR
1.000000e+10    4527
Name: count, dtype: int64

### Dim Time

In [81]:
dim_time = total_exports_clean[["FECHA_DECLARACION_EXPORTACION"]].drop_duplicates().copy()
dim_time["FECHA_DECLARACION_EXPORTACION"] = pd.to_datetime(dim_time["FECHA_DECLARACION_EXPORTACION"], format="%Y%m%d")
dim_time["anio"] = dim_time["FECHA_DECLARACION_EXPORTACION"].dt.year
dim_time["mes"] = dim_time["FECHA_DECLARACION_EXPORTACION"].dt.month
dim_time["dia"] = dim_time["FECHA_DECLARACION_EXPORTACION"].dt.day
dim_time = dim_time[["FECHA_DECLARACION_EXPORTACION", "anio", "mes", "dia"]]
dim_time.sort_values("FECHA_DECLARACION_EXPORTACION", inplace=True)
dim_time = dim_time.rename(columns={"FECHA_DECLARACION_EXPORTACION": "id_tiempo"})
dim_time = dim_time.reset_index(drop=True)
dim_time

,id_tiempo,anio,mes,dia
0,2025-01-01,2025,1,1
1,2025-01-02,2025,1,2
2,2025-01-03,2025,1,3
3,2025-01-04,2025,1,4
4,2025-01-05,2025,1,5
...,...,...,...,...
268,2025-09-26,2025,9,26
269,2025-09-27,2025,9,27
270,2025-09-28,2025,9,28
271,2025-09-29,2025,9,29


### Dim Empresa

In [116]:
dim_empresa = total_exports_clean[["NIT_EXPORTADOR", "TIPO_USUARIO", "COD_USUARIO", "CLASE_EXPORTADOR", "COD_DPTO_EXPORTADOR", "RAZON_SOCIAL_EXPORTADOR", "DIREC_EXPORTADOR"]].drop_duplicates().copy()
dim_empresa = dim_empresa.reset_index(drop=True)
dim_empresa = dim_empresa.rename(columns={"NIT_EXPORTADOR": "nit", "TIPO_USUARIO": "tipo_usuario", "COD_USUARIO": "cod_usuario", "CLASE_EXPORTADOR": "clase", "COD_DPTO_EXPORTADOR": "cod_departamento", "RAZON_SOCIAL_EXPORTADOR": "razon_social", "DIREC_EXPORTADOR": "direccion"})
dim_empresa = dim_empresa.drop_duplicates(subset=["nit"], keep="first").copy()
dim_empresa["nit"] = dim_empresa["nit"].astype(float).astype("Int64").astype(str)
dim_empresa

,nit,tipo_usuario,cod_usuario,clase,cod_departamento,razon_social,direccion
0,900647398,36,0,2,76001,THE PREMIER TRADING CI SAS,CL 20 NORTE 5 B N 38 OF 301
1,860026753,36,0,2,11001,ACESCO COLOMBIA S.A.S.,AV CL 116 7 15 IN 2 P 17 OF 1701
2,860001899,36,0,2,11001,CORPACERO S. A. S.,CL 25 G 100 24
3,901436775,36,0,2,11001,FABRICA DE LICORES Y ALCOHOLES DE ANTIOQUIA,AC 24 96 10 BG 2 OF 303
4,860402328,36,0,2,11001,INTECPLAST INYECCION TECNICA DE PLASTICOS S. A...,AC 24 95 12 BG 16
...,...,...,...,...,...,...,...
13823,901053611,36,0,2,8001,COMERCIALIZADORA GOLDEN FOOD S.A.S.,CR 53 76 239 OF 302
13825,805014201,36,0,2,5266,CALDERAS DEL OCCIDENTE S. A. S.,CL 23 SUR 42 B 60 OF 201 ED O'CLOCK
13826,901134015,36,0,2,54001,TUBOS DEL ORIENTE TUBERIA Y ACCESORIOS S.A.S.,AV 7 21 N 55 OF 406 ED ESTACION CENTRO ...
13829,830056873,36,0,2,11001,COMERCIALIZADORA TRACTO LASER SAS,CL 25 B 81 36 P 1 BRR MODELIA


### Dim Transporte

In [100]:
dim_transporte = total_exports_clean[["COD_MODO_TRANSPORTE", "MODO_TRANSPORTE"]].drop_duplicates().copy()
dim_transporte = dim_transporte.reset_index(drop=True)
dim_transporte = dim_transporte.rename(columns={"COD_MODO_TRANSPORTE": "cod_modo_transporte", "MODO_TRANSPORTE": "modo_transporte"})
dim_transporte

,cod_modo_transporte,modo_transporte
0,4,Aéreo
1,1,Marítimo
2,7,Instalaciones Fijas
3,3,Terrestre (carretero)
4,8,Vías navegables interiores


### Dim Aduanas

In [71]:
dim_aduanas = total_exports_clean[["COD_ADUANA_DESPACHO", "ADUANA_DESPACHO"]].drop_duplicates().copy()
dim_aduanas = dim_aduanas.reset_index(drop=True)
dim_aduanas = dim_aduanas.rename(columns={"COD_ADUANA_DESPACHO": "cod_aduana_despacho", "ADUANA_DESPACHO": "aduana_despacho"})
dim_aduanas

,cod_aduana_despacho,aduana_despacho
0,3,Aduanas de Bogotá - Aeropuerto El Dorado
1,87,Aduanas de Barranquilla
2,48,Aduanas de Cartagena
3,19,Impuestos y Aduanas de Santa Marta
4,90,Aduanas de Medellín
5,41,Impuestos y Aduanas de Urabá
6,35,Impuestos y Aduanas de Buenaventura
7,25,Impuestos y Aduanas de Riohacha
8,88,Aduanas de Cali
9,37,Impuestos y Aduanas de Ipiales


### Dim Destino

- Worldcities Dataset para extraer **LATITUD y LONGITUD**

In [41]:
worldcities = pd.read_csv("./Datasets/worldcities/worldcities.csv")
worldcities_subset = worldcities[["city", "lat", "lng", "iso3"]].copy()
worldcities_subset["city"] = worldcities_subset["city"].str.upper().str.strip()
worldcities_subset["iso3"] = worldcities_subset["iso3"].str.upper().str.strip()

world_latlng_by_country = worldcities_subset.groupby("iso3")[["lat", "lng"]].mean().reset_index().rename(columns={"iso3": "COD_PAIS_DESTINO", "lat": "LATITUD", "lng": "LONGITUD"})

In [54]:
dim_destino = total_exports_clean[["ID_DESTINO", "COD_PAIS_DESTINO", "PAIS_DESTINO_FINAL", "CIUDAD_DESTINATARIO"]].drop_duplicates().copy()
dim_destino = dim_destino.reset_index(drop=True)
dim_destino_geo = dim_destino.merge(world_latlng_by_country, on="COD_PAIS_DESTINO", how="left")
dim_destino_geo.isnull().sum()

ID_DESTINO             0
COD_PAIS_DESTINO       0
PAIS_DESTINO_FINAL     0
CIUDAD_DESTINATARIO    0
LATITUD                2
LONGITUD               2
dtype: int64

In [ ]:
faltantes_geo = dim_destino[dim_destino_geo["LATITUD"].isna() | dim_destino_geo["LONGITUD"].isna()]
faltantes_geo

,ID_DESTINO,COD_PAIS_DESTINO,PAIS_DESTINO_FINAL,CIUDAD_DESTINATARIO
2342,PSE_HEBRON,PSE,Estado de Palestina,HEBRON
4871,IOT_PROVIDENCIALES,IOT,Territorio Británico del Océano Índico,PROVIDENCIALES


In [56]:
# Hebrón (PSE)
mask_pse = (dim_destino_geo["COD_PAIS_DESTINO"]=="PSE") & (dim_destino_geo["CIUDAD_DESTINATARIO"].str.upper().str.contains("HEBRON"))
dim_destino_geo.loc[mask_pse, ["LATITUD","LONGITUD"]] = [31.5294, 35.0938]

# Providenciales (IOT)
mask_iot = (dim_destino_geo["COD_PAIS_DESTINO"]=="IOT") & (dim_destino_geo["CIUDAD_DESTINATARIO"].str.upper().str.contains("PROVIDENCIALES"))
dim_destino_geo.loc[mask_iot, ["LATITUD","LONGITUD"]] = [-7.3195, 72.422859]

dim_destino_geo.isnull().sum()

ID_DESTINO             0
COD_PAIS_DESTINO       0
PAIS_DESTINO_FINAL     0
CIUDAD_DESTINATARIO    0
LATITUD                0
LONGITUD               0
dtype: int64

In [118]:
dim_destino_geo = dim_destino_geo.rename(columns={"ID_DESTINO": "id_destino", "COD_PAIS_DESTINO": "cod_pais_destino", "PAIS_DESTINO_FINAL": "pais_destino_final", "CIUDAD_DESTINATARIO": "ciudad_destinatario", "LATITUD": "latitud", "LONGITUD": "longitud"})
dim_destino_geo

,id_destino,cod_pais_destino,pais_destino_final,ciudad_destinatario,latitud,longitud
0,HKG_KOWLOON,HKG,"China, región administrativa especial de Hong ...",KOWLOON,22.334791,114.149043
1,BRB_CHRIST_CHURCH,BRB,Barbados,CHRIST CHURCH,13.096900,-59.613100
2,MEX_MONTERREY,MEX,México,MONTERREY,20.551622,-99.328809
3,USA_MIAMI,USA,Estados Unidos de América,MIAMI,38.142564,-90.212717
4,USA_GAINESVILLE,USA,Estados Unidos de América,GAINESVILLE,38.142564,-90.212717
...,...,...,...,...,...,...
10188,VEN_CHURURU,VEN,Venezuela (República Bolivariana de),CHURURU,9.574322,-66.576424
10189,VEN_LA_TENDIDA,VEN,Venezuela (República Bolivariana de),LA TENDIDA,9.574322,-66.576424
10190,VEN_EL_NULA_EDO_APURE,VEN,Venezuela (República Bolivariana de),EL NULA EDO APURE,9.574322,-66.576424
10191,VEN_MARIARA,VEN,Venezuela (República Bolivariana de),MARIARA,9.574322,-66.576424


### Dim Modalidad

In [73]:
dim_modalidad = total_exports_clean[["COD_MODALIDAD_EXPORTACION", "MODALIDAD_EXPORTACION"]].drop_duplicates().copy()
dim_modalidad = dim_modalidad.reset_index(drop=True)
dim_modalidad = dim_modalidad.rename(columns={"COD_MODALIDAD_EXPORTACION": "cod_modalidad_exportacion", "MODALIDAD_EXPORTACION": "modalidad_exportacion"})
dim_modalidad

,cod_modalidad_exportacion,modalidad_exportacion
0,199,Las demás exportaciones definitivas no incluid...
1,198,Exportación definitiva de mercancías de fabric...
2,104,Exportación definitiva de mercancías que resul...
3,107,Donaciones
4,2,Muestras sin valor comercial
5,4,Menaje
6,201,Mercancías exportadas temporalmente para trans...
7,401,Reexportación definitiva de mercancías que est...
8,402,Reexportación definitiva de mercancías import...
9,403,Reexportación temporal de bienes de capital o ...


### Fact Exportaciones

In [130]:
fact_exportaciones = total_exports_clean[[
    "UNIDAD_FISICA",
    "CANTIDAD_UNIDADES_FISICAS",
    "PESO_BRUTO_KGS",
    "PESO_NETO_KGS",
    "VALOR_FOB_USD",
    "VALOR_FOB_PESOS",
    "VLR_SERIE_AGREGADO_NAL_USD",
    "VALOR_SERIE_FLETES_USD",
    "VALOR_SERIE_SEGUROS_USD",
    "VLR_SERIE_OTROS_GASTOS_USD",
    "COD_MONEDA_TRANSACCION",
    "COD_MODO_TRANSPORTE",
    "COD_MODALIDAD_EXPORTACION",
    "NIT_EXPORTADOR",
    "COD_ADUANA_DESPACHO",
    "FECHA_DECLARACION_EXPORTACION",
    "ID_DESTINO"
]].copy()

In [131]:
fact_exportaciones["ID_EXPORTACION"] = [uuid.uuid4().hex for _ in range(len(fact_exportaciones))]

In [132]:
fact_exportaciones = fact_exportaciones[[
    "ID_EXPORTACION",
    "NIT_EXPORTADOR",
    "COD_ADUANA_DESPACHO",
    "COD_MODO_TRANSPORTE",
    "COD_MODALIDAD_EXPORTACION",
    "FECHA_DECLARACION_EXPORTACION",
    "ID_DESTINO",
    "UNIDAD_FISICA",
    "CANTIDAD_UNIDADES_FISICAS",
    "PESO_BRUTO_KGS",
    "PESO_NETO_KGS",
    "VALOR_FOB_USD",
    "VALOR_FOB_PESOS",
    "VLR_SERIE_AGREGADO_NAL_USD",
    "VALOR_SERIE_FLETES_USD",
    "VALOR_SERIE_SEGUROS_USD",
    "VLR_SERIE_OTROS_GASTOS_USD",
    "COD_MONEDA_TRANSACCION"
]]
fact_exportaciones = fact_exportaciones.rename(columns={
    "ID_EXPORTACION": "id_exportacion",
    "NIT_EXPORTADOR": "nit",
    "COD_ADUANA_DESPACHO": "cod_aduana_despacho",
    "COD_MODO_TRANSPORTE": "cod_modo_transporte",
    "COD_MODALIDAD_EXPORTACION": "cod_modalidad_exportacion",
    "FECHA_DECLARACION_EXPORTACION": "id_tiempo",
    "ID_DESTINO": "id_destino",
    "UNIDAD_FISICA": "unidad_fisica",
    "CANTIDAD_UNIDADES_FISICAS": "cantidad_unidades_fisicas",
    "PESO_BRUTO_KGS": "peso_bruto_kgs",
    "PESO_NETO_KGS": "peso_neto_kgs",
    "VALOR_FOB_USD": "valor_fob_usd",
    "VALOR_FOB_PESOS": "valor_fob_pesos",
    "VLR_SERIE_AGREGADO_NAL_USD": "vlr_serie_agregado_nal_usd",
    "VALOR_SERIE_FLETES_USD": "valor_serie_fletes_usd",
    "VALOR_SERIE_SEGUROS_USD": "valor_serie_seguros_usd",
    "VLR_SERIE_OTROS_GASTOS_USD": "valor_serie_otros_gastos_usd",
    "COD_MONEDA_TRANSACCION": "moneda_transaccion"
})
fact_exportaciones["nit"] = fact_exportaciones["nit"].astype(float).astype("Int64").astype(str)
fact_exportaciones

,id_exportacion,nit,cod_aduana_despacho,cod_modo_transporte,cod_modalidad_exportacion,id_tiempo,id_destino,unidad_fisica,cantidad_unidades_fisicas,peso_bruto_kgs,peso_neto_kgs,valor_fob_usd,valor_fob_pesos,vlr_serie_agregado_nal_usd,valor_serie_fletes_usd,valor_serie_seguros_usd,valor_serie_otros_gastos_usd,moneda_transaccion
0,b856fc83189b46b09540c73ad170a3ca,900647398,3,4,199,2025-01-24,HKG_KOWLOON,Quilate,145.55,0.03,0.02,108800.00,4.619267e+08,0.00,0.00,0.0,0.00,USD
1,7ce784d92ac9498b87e2dd3db8cfea4b,860026753,87,1,199,2025-01-10,BRB_CHRIST_CHURCH,Kilogramo,42734.00,43119.00,42734.00,44379.79,1.917735e+08,0.00,6260.00,0.0,0.00,USD
2,f08042164a834def855582d59ab03ae9,860001899,87,1,198,2025-01-23,MEX_MONTERREY,Kilogramo,1014460.00,1017700.00,1014460.00,999243.10,4.274772e+09,294083.13,95694.01,119.0,0.00,USD
3,4f18b128c9f0470e9836f56a8d8f1b1b,901436775,48,1,104,2025-01-08,USA_MIAMI,Litro,189.00,196.07,189.00,927.36,4.026885e+06,0.00,0.00,0.0,0.00,USD
4,956e32d480924d3c8e88d1daf4524a7d,860402328,48,1,198,2025-01-07,USA_MIAMI,Unidades o artículos,116831.00,2473.16,1967.96,13301.52,5.793490e+07,0.00,0.00,0.0,0.00,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121421,12944925ae4b42f5853094f34afd067f,890807976,89,3,198,2025-09-05,VEN_CARABOBO,Unidades o artículos,144.00,45.75,44.00,343.35,1.370341e+06,0.00,16.65,0.0,0.00,USD
121422,be6891a1abc74964bd69621cc7b79f86,890807976,89,3,198,2025-09-05,VEN_CARABOBO,Unidades o artículos,192.00,57.19,55.00,423.00,1.688231e+06,0.00,20.52,0.0,0.00,USD
121423,ed5209dd75354b3c9acc665e6b3595c4,900617753,39,3,198,2025-09-19,VEN_CARACAS,Unidades o artículos,1.00,22.45,14.88,268.40,1.044734e+06,0.00,0.00,0.0,24.16,USD
121424,c40c6a1603c14fed899d503d772b78cd,900617753,39,3,198,2025-09-19,VEN_CARACAS,Unidades o artículos,219.00,1195.34,791.99,14281.30,5.558925e+07,0.00,0.00,0.0,1285.32,USD


## Carga

In [78]:
load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [ ]:
dim_time.to_sql("dim_time", engine, if_exists="append", index=False)
dim_empresa.to_sql("dim_empresa", engine, if_exists="append", index=False)
dim_transporte.to_sql("dim_transporte", engine, if_exists="append", index=False)
dim_aduanas.to_sql("dim_aduanas", engine, if_exists="append", index=False)
dim_destino_geo.to_sql("dim_destino", engine, if_exists="append", index=False)
dim_modalidad.to_sql("dim_modalidad", engine, if_exists="append", index=False)
fact_exportaciones.to_sql("fact_exportaciones", engine, if_exists="append", index=False, chunksize=50000)

20592